## Data from API

Connect to the SF.Gov API and pull data from the 311 Table
https://data.sfgov.org/City-Infrastructure/Case-Data-from-San-Francisco-311-SF311-/vw6y-z8j6

Note: You'll need to sign up for an app token to avoid throttling limits http://dev.socrata.com/foundry/#/data.sfgov.org/vw6y-z8j6

hint: use the built in JSON functionality inside python

In [7]:
import json
import urllib
import pandas as pd
import sqlite3

In [8]:
url = "https://data.sfgov.org/resource/vw6y-z8j6.json?$$app_token=Iq3JEcmMzuOIvahirX7cR0GTI"
data = json.load(urllib.urlopen(url))

### How many elements are there in the JSON file? Display the length of the JSON object and display the elements contained in one of the items of the JSON file?

In [3]:
data[0]

{u'address': u'Intersection of 21ST ST and CAPP ST',
 u'case_id': u'322571',
 u'category': u'General Requests',
 u'closed': u'2008-12-01T15:19:00',
 u'neighborhood': u'Mission',
 u'opened': u'2008-11-30T23:53:00',
 u'point': {u'latitude': u'37.7571008516766',
  u'longitude': u'-122.417811874214',
  u'needs_recoding': False},
 u'request_details': u'puc - electric - request_for_service',
 u'request_type': u'puc - electric - request_for_service',
 u'responsible_agency': u'PUC - Electric/Power - G',
 u'source': u'Voice In',
 u'status': u'Closed',
 u'supervisor_district': u'9',
 u'updated': u'2008-12-01T15:19:00'}

### Use Pandas "Read JSON" functionality to turn the 311 data into a Pandas Dataframe

In [10]:
df = pd.DataFrame(data)
df.head(5)

,address,case_id,category,closed,neighborhood,opened,point,request_details,request_type,responsible_agency,source,status,supervisor_district,updated
0,Intersection of 21ST ST and CAPP ST,322571,General Requests,2008-12-01T15:19:00,Mission,2008-11-30T23:53:00,"{u'latitude': u'37.7571008516766', u'needs_rec...",puc - electric - request_for_service,puc - electric - request_for_service,PUC - Electric/Power - G,Voice In,Closed,9,2008-12-01T15:19:00
1,Intersection of BUSH ST and VAN NESS AVE,322568,Illegal Postings,2009-07-21T16:24:00,Nob Hill,2008-11-30T23:13:00,"{u'latitude': u'37.7884895281133', u'needs_rec...",Posting_Too_Large_in_Size on Sidewalk,Illegal Postings - Posting_Too_Large_in_Size,DPW Ops Queue,Voice In,Closed,3,2009-07-21T16:24:00
2,Intersection of EUCLID AVE and MASONIC AVE,322567,Illegal Postings,2008-12-27T06:07:00,Western Addition,2008-11-30T23:07:00,"{u'latitude': u'37.7850837365507', u'needs_rec...",Affixed_Improperly on Sidewalk,Illegal Postings - Affixed_Improperly,DPW Ops Queue,Voice In,Closed,2,2008-12-27T06:07:00
3,"1566 HYDE ST, SAN FRANCISCO, CA, 94109",322566,Street and Sidewalk Cleaning,2009-07-21T16:24:00,Nob Hill,2008-11-30T22:56:00,"{u'latitude': u'37.795328529', u'needs_recodin...",Garbage,Sidewalk_Cleaning,DPW Ops Queue,Voice In,Closed,3,2009-07-21T16:24:00
4,"GGP Panhandle, SAN FRANCISCO, CA, 94117",322565,Rec and Park Requests,2008-12-13T10:50:00,Haight Ashbury,2008-11-30T22:46:00,"{u'latitude': u'37.772204762', u'needs_recodin...",Other,Park - Structural_Maintenance,RPD Park Service Area GGP Queue,Voice In,Closed,5,2008-12-13T10:50:00


In [13]:
del df['point']

### Create a Databse and Store the results.

Here, we are creating the DB in memory. You can change it to a real DB filename if you want to keep it for further analysis

In [14]:
# Write to Database.  We will then use to run queries
con = sqlite3.connect(":memory:")
df.to_sql('sf_311', con)

### Display first 5 rows

In [16]:
pd.read_sql("SELECT * FROM sf_311 limit 5", con)

,index,address,case_id,category,closed,neighborhood,opened,request_details,request_type,responsible_agency,source,status,supervisor_district,updated
0,0,Intersection of 21ST ST and CAPP ST,322571,General Requests,2008-12-01T15:19:00,Mission,2008-11-30T23:53:00,puc - electric - request_for_service,puc - electric - request_for_service,PUC - Electric/Power - G,Voice In,Closed,9,2008-12-01T15:19:00
1,1,Intersection of BUSH ST and VAN NESS AVE,322568,Illegal Postings,2009-07-21T16:24:00,Nob Hill,2008-11-30T23:13:00,Posting_Too_Large_in_Size on Sidewalk,Illegal Postings - Posting_Too_Large_in_Size,DPW Ops Queue,Voice In,Closed,3,2009-07-21T16:24:00
2,2,Intersection of EUCLID AVE and MASONIC AVE,322567,Illegal Postings,2008-12-27T06:07:00,Western Addition,2008-11-30T23:07:00,Affixed_Improperly on Sidewalk,Illegal Postings - Affixed_Improperly,DPW Ops Queue,Voice In,Closed,2,2008-12-27T06:07:00
3,3,"1566 HYDE ST, SAN FRANCISCO, CA, 94109",322566,Street and Sidewalk Cleaning,2009-07-21T16:24:00,Nob Hill,2008-11-30T22:56:00,Garbage,Sidewalk_Cleaning,DPW Ops Queue,Voice In,Closed,3,2009-07-21T16:24:00
4,4,"GGP Panhandle, SAN FRANCISCO, CA, 94117",322565,Rec and Park Requests,2008-12-13T10:50:00,Haight Ashbury,2008-11-30T22:46:00,Other,Park - Structural_Maintenance,RPD Park Service Area GGP Queue,Voice In,Closed,5,2008-12-13T10:50:00


### Which type of complaint was most common?

### Show 10 of the sidewalk cleanings (request_type='Sidewalk_Cleaning') in the Haight Ashbury neighborhood

Hint:
* request_type='Sidewalk_Cleaning'
* neighborhood='Haight Ashbury'

### Are there any neighborhoods where sidewalk cleaning requests are the most common?